#Twitter Sentimental Analysis using Tone Analyzer 
Loading the data
Here, I am loading the custom library generate as part of the Twitter streaming application with Tone Analyzer service.
The following code is using a pre-built jar has been posted on the Github project at given repository.

In [1]:
%AddJar https://github.com/ibm-cds-labs/spark.samples/raw/master/dist/streaming-twitter-assembly-1.2.jar -f

Starting download from https://github.com/ibm-cds-labs/spark.samples/raw/master/dist/streaming-twitter-assembly-1.2.jar
Finished download of streaming-twitter-assembly-1.2.jar


#Setting up the Twitter and Tone Analyzer credentials
Go to the service credential tab of Tone Analyzer service to get Username and Password. Twitter credentials are described in detailed in project report.

In [2]:
val demo = com.ibm.cds.spark.samples.StreamingTwitter
demo.setConfig("twitter4j.oauth.consumerKey","xxxxxxxxxxxxxxxxxxxxxxxxxx")
demo.setConfig("twitter4j.oauth.consumerSecret","xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxn")
demo.setConfig("twitter4j.oauth.accessToken","xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
demo.setConfig("twitter4j.oauth.accessTokenSecret","xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
demo.setConfig("watson.tone.url","https://gateway.watsonplatform.net/tone-analyzer-experimental/api")
demo.setConfig("watson.tone.password","xxxxxxxxxxxxxxxx")
demo.setConfig("watson.tone.username","xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

#Starting the Spark Stream to get live tweets by providing timeinterval
Start a new Twitter Stream that collects the live tweets and enrich them with Sentiment Analysis scores. 

The stream is run for a duration specified in the second argument of the **startTwitterStreaming** method.

Note: if no duration is specified then the stream will run until the **stopTwitterStreaming** method is called.

In [3]:
import org.apache.spark.streaming._
demo.startTwitterStreaming(sc, Seconds(60))

Twitter stream started
Tweets are collected real-time and analyzed
To stop the streaming and start interacting with the data use: StreamingTwitter.stopTwitterStreaming
Stopping Twitter stream. Please wait this may take a while
Twitter stream stopped
You can now create a sqlContext and DataFrame with 606 Tweets created. Sample usage: 
val (sqlContext, df) = com.ibm.cds.spark.samples.StreamingTwitter.createTwitterDataFrames(sc)
df.printSchema
sqlContext.sql("select author, text from tweets").show


#Creating a SQLContext and Dataframe with all the tweets
Note: this method will register a SparkSQL table called tweets

In [4]:
val (sqlContext, df) = demo.createTwitterDataFrames(sc)

A new table named tweets with 606 records has been correctly created and can be accessed through the SQLContext variable
Here's the schema for tweets
root
 |-- author: string (nullable = true)
 |-- date: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- text: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- Cheerfulness: double (nullable = true)
 |-- Negative: double (nullable = true)
 |-- Anger: double (nullable = true)
 |-- Analytical: double (nullable = true)
 |-- Confident: double (nullable = true)
 |-- Tentative: double (nullable = true)
 |-- Openness: double (nullable = true)
 |-- Agreeableness: double (nullable = true)
 |-- Conscientiousness: double (nullable = true)



#Executing a SparkSQL query to display the data, table has collected

In [5]:
val fullSet = sqlContext.sql("select * from tweets")  //Select all columns

In [6]:
fullSet.show    //to display data in fullSet

+-----------------+--------------------+-----+--------------------+--------+---------+------------+--------+-----+----------+---------+---------+------------------+-------------+------------------+
|           author|                date| lang|                text|     lat|     long|Cheerfulness|Negative|Anger|Analytical|Confident|Tentative|          Openness|Agreeableness| Conscientiousness|
+-----------------+--------------------+-----+--------------------+--------+---------+------------+--------+-----+----------+---------+---------+------------------+-------------+------------------+
|            Myles|Fri Dec 11 18:37:...|   en|@MadSports8 @SKar...|     0.0|      0.0|         0.0|     0.0|  0.0|       0.0|      0.0|    100.0|               9.0|          1.0| 7.000000000000001|
|           isabel|Fri Dec 11 18:37:...|   en|@trulyoth jk my r...|     0.0|      0.0|         0.0|     0.0|  0.0|      86.0|      0.0|      0.0|               0.0|        100.0|               2.0|
|    TTN C

#Persist the dataset into a parquet file on Object Storage service
The parquet file will be reloaded here

In [7]:
fullSet.repartition(1).saveAsParquetFile("swift://notebooks.spark/tweetsFull.parquet")   //saving data into a parquet file... 

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


#Running SparkSQL query on the data collceted
Lets, select all the tweets that have Anger score greated than 70%

In [8]:
val angerSet = sqlContext.sql("select author, text, Anger from tweets where Anger > 70")

In [9]:
println(angerSet.count)   // check count of angerSet here

40


In [10]:
angerSet.show   //display all data

+--------------------+--------------------+-----+
|              author|                text|Anger|
+--------------------+--------------------+-----+
|    Bussy The Bunny?|RT @KEEPINUPWITKE...|100.0|
|          Bulls 74-8|RT @MachineGunnKi...|100.0|
|               daddy|@StarsAndScars_ 
...|100.0|
|    Fade VermaVirus |My friend got mad...|100.0|
|     Zachary Daniels|RT @nxlix_: make ...|100.0|
|                 BFD|Yes @celestepewte...|100.0|
|     Jonny Pinciotti|                Shit|100.0|
|        jolly jazmin|my dad fucking te...|100.0|
|                Alan|@AbiRatchford GOD...|100.0|
|            Lindsay?|Hate myself more ...|100.0|
|     Ashley Reynolds|RT @LoveIyCouples...|100.0|
|         Tamara King|lovepleasure0907:...|100.0|
|           anggi_Dot|2 weekly follower...|100.0|
|aggressive zoom blur|@tarotots arbuckl...|100.0|
|                 Jrd|@DieErVonSatan yo...|100.0|
|                  SB|I need all that shit|100.0|
|              Ethan.|EVER SEEN SOMEONE...|100.0|
